In [23]:
import os
import pandas as pd
import numpy as np
import warnings
from datetime import datetime
from zipline.utils.factory import load_bars_from_yahoo
import pytz
import numpy as np
from sklearn import model_selection
from sklearn import preprocessing
import matplotlib.pyplot as plt
import ystockquote

In [29]:
symbols = ['A', 'AA', 'AAP', 'ABBV', 'ABT', 'ACN', 'ADBE', 'AES', 'AET', 'AFL']
start = datetime(2000, 1, 1, 0, 0, 0, 0, pytz.utc)
end = datetime(2016, 4, 30, 0, 0, 0, 0, pytz.utc)
dates = pd.date_range(start,end)

In [30]:
dates

DatetimeIndex(['2000-01-01', '2000-01-02', '2000-01-03', '2000-01-04',
               '2000-01-05', '2000-01-06', '2000-01-07', '2000-01-08',
               '2000-01-09', '2000-01-10',
               ...
               '2016-04-21', '2016-04-22', '2016-04-23', '2016-04-24',
               '2016-04-25', '2016-04-26', '2016-04-27', '2016-04-28',
               '2016-04-29', '2016-04-30'],
              dtype='datetime64[ns, UTC]', length=5965, freq='D')

In [32]:
dates = pd.date_range('2011-10-13', '2016-10-12')
dates

DatetimeIndex(['2011-10-13', '2011-10-14', '2011-10-15', '2011-10-16',
               '2011-10-17', '2011-10-18', '2011-10-19', '2011-10-20',
               '2011-10-21', '2011-10-22',
               ...
               '2016-10-03', '2016-10-04', '2016-10-05', '2016-10-06',
               '2016-10-07', '2016-10-08', '2016-10-09', '2016-10-10',
               '2016-10-11', '2016-10-12'],
              dtype='datetime64[ns]', length=1827, freq='D')

In [33]:
def get_data(symbols, dates):
    df = pd.DataFrame(index=dates)
    data = []
    for symbol in symbols:
        df_temp = pd.read_csv(os.path.join("Data/table_{}.csv".format(str(symbol))), index_col='Date',
                              parse_dates=True, usecols=['Date', 'Adj Close'], na_values=['nan'])
        df_comp = pd.read_csv(os.path.join("Data/table_{}.csv".format(str(symbol))), index_col='Date',
                              parse_dates=True, na_values=['nan'])
        data.append(df_comp)
        df_temp = df_temp.rename(columns={'Adj Close': symbol})
        df = df.join(df_temp)
    return [df, data]

In [34]:
[df, data] = get_data(symbols, dates)

In [35]:
data[1]

,Open,High,Low,Close,Volume,Adj Close
Date,,,,,,
2016-10-12,27.520000,27.520000,27.020000,27.110001,19056200,27.110001
2016-10-11,29.730000,29.730000,27.889999,27.910000,32668200,27.910000
2016-10-10,31.570000,31.959999,31.370001,31.510000,9119000,31.510000
2016-10-07,32.020000,32.099998,31.030001,31.370001,7858600,31.370001
2016-10-06,31.070000,32.020000,31.070000,31.780001,10026500,31.780001
2016-10-05,10.249999,10.410000,10.170000,10.400000,29736800,31.199999
2016-10-04,10.059990,10.209989,9.989989,10.199989,10663500,30.599968
2016-10-03,10.139989,10.159989,9.959989,10.119990,5460900,30.359969
2016-09-30,9.979990,10.189989,9.949990,10.139989,7629600,30.419968


In [44]:
jso= ystockquote.get_historical_prices('GOOGL', '2011-10-13', '2016-10-12')

In [63]:
jso

{'2014-06-30': {'Adj Close': '584.669983',
  'Close': '584.669983',
  'High': '587.710022',
  'Low': '583.289978',
  'Open': '586.76001',
  'Volume': '1478900'},
 '2014-10-16': {'Adj Close': '536.919983',
  'Close': '536.919983',
  'High': '540.98999',
  'Low': '524.950012',
  'Open': '527.00',
  'Volume': '3805000'},
 '2011-12-15': {'Adj Close': '310.080082',
  'Close': '619.540004',
  'High': '624.000017',
  'Low': '618.690025',
  'Open': '622.519992',
  'Volume': '4812300'},
 '2015-11-19': {'Adj Close': '759.940002',
  'Close': '759.940002',
  'High': '763.22998',
  'Low': '757.22998',
  'Open': '759.780029',
  'Volume': '1403900'},
 '2015-11-18': {'Adj Close': '760.01001',
  'Close': '760.01001',
  'High': '761.099976',
  'Low': '747.390015',
  'Open': '749.570007',
  'Volume': '1813300'},
 '2011-11-04': {'Adj Close': '298.368382',
  'Close': '596.140026',
  'High': '599.739999',
  'Low': '592.429995',
  'Open': '593.50003',
  'Volume': '5663900'},
 '2013-06-19': {'Adj Close': '450

In [67]:
pd.read_json(str(jso),orient='index')

ValueError: Expected object or value

In [68]:
s='{'2014-06-30': {\'Adj Close\': \'584.669983\',
  'Close': '584.669983',
  'High': '587.710022',
  'Low': '583.289978',
  'Open': '586.76001',
  'Volume': '1478900'}'

SyntaxError: invalid syntax (<ipython-input-68-70bf65c4895f>, line 1)